### Libraries

In [93]:
from IPython.display import Audio
from scipy.io import wavfile
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np

audio_file_path='C:\MBA\project\data\Speech_4.wav'
samplerate, tabulasa = wavfile.read(audio_file_path)



### Main Funcrion

In [88]:

def pca_reduce(signal, n_components, block_size=1024):
    
    # First, zero-pad the signal so that it is divisible by the block_size
    samples = len(signal)
    hanging = block_size - np.mod(samples, block_size)
    padded = np.lib.pad(signal, (0, hanging), 'constant', constant_values=0)
    
    # Reshape the signal to have 1024 dimensions
    reshaped = padded.reshape((len(padded) // block_size, block_size))
    
    # Second, do the actual PCA process
    pca = PCA(n_components=n_components)
    pca.fit(reshaped)
    
    transformed = pca.transform(reshaped)
    reconstructed = pca.inverse_transform(transformed).reshape((len(padded)))
    return pca, transformed, reconstructed

### Convert to MB

In [66]:
def raw_estimate(transformed):
    # We assume that we'll be storing things as 16-bit WAV,
    # meaning two bytes per sample
    t_size = transformed.nbytes
    
    # Return a result in megabytes
    return t_size/ (2**20)

### Iteration

In [67]:
compression_attempts = [
    (1, 1),
    (1, 2),
    (1, 4),
    (4, 32),
    (16, 256),
    (32, 256),
    (64, 256),
    (128, 1024),
    (4, 2048),
    (16, 2048),
    (32, 2048),
    (64, 2048)
]

### Main pipeline

In [68]:
def build_estimates(signal, n_components, block_size):
    pca, transformed, recon = pca_reduce(signal, n_components, block_size)
    raw_pca_estimate = raw_estimate(transformed)
    raw_size = recon.nbytes / (2**20)
    return raw_size, raw_pca_estimate

In [69]:
pca_compression_results = pd.DataFrame([
        build_estimates(tabulasa, n, bs)
        for n, bs in compression_attempts
    ])

In [70]:
pca_compression_results.columns = ["Raw", "PCA"]
pca_compression_results.index = compression_attempts
pca_compression_results

,Raw,PCA
"(1, 1)",1.007141,1.007141
"(1, 2)",1.007141,0.503571
"(1, 4)",1.007141,0.251785
"(4, 32)",1.007324,0.125916
"(16, 256)",1.007812,0.062988
"(32, 256)",1.007812,0.125977
"(64, 256)",1.007812,0.251953
"(128, 1024)",1.007812,0.125977
"(4, 2048)",1.015625,0.001984
"(16, 2048)",1.015625,0.007935


In [74]:
_,_,reconstructed_4_32 = pca_reduce(tabulasa,4,32)

In [75]:
Audio(data=reconstructed_4_32, rate=samplerate)

In [78]:
_,_,reconstructed_64_256 = pca_reduce(tabulasa,64,256)

In [79]:
Audio(data=reconstructed_64_256, rate=samplerate)

In [81]:
_,_,reconstructed_1028 = pca_reduce(tabulasa,128,1024)

In [82]:
Audio(data=reconstructed_1028, rate=samplerate)

In [83]:
_,_,reconstructed_2048 = pca_reduce(tabulasa,64,2048)

In [84]:
Audio(data=reconstructed_2048, rate=samplerate)